# Funções para geração de resultados

In [2]:
varia_filmes = [1000, 10000, 100000, 1000000, 10000000]
varia_categorias = [1, 2, 3, 4, 5, 6]

for i in varia_filmes:
    for j in varia_categorias:
        !./geradorInputs {i} {j}
        !mv input.txt in-{i}-{j}.txt